<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
#load relevant packages
import scanpy as sc
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import sys
import os
import pandas as pd
import besca as bc

#setup document
sc.settings.verbosity = 3  # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.settings.set_figure_params(dpi=90)  # low dpi (dots per inch) yields small inline figures
sc.logging.print_versions()

sc.settings.figdir = './results/'
sns.set_style('ticks')
plt.rcParams['image.cmap']='viridis'
plt.rcParams['svg.fonttype'] = 'none'

/usr/local/anaconda3/envs/besca/lib/python3.7/site-packages/anndata/_core/anndata.py:21: FutureWarning: pandas.core.index is deprecated and will be removed in a future version.  The public classes are available in the top-level namespace.
  from pandas.core.index import RangeIndex
/usr/local/anaconda3/envs/besca/lib/python3.7/site-packages/scanpy/api/__init__.py:6: FutureWarning: 

In a future version of Scanpy, `scanpy.api` will be removed.
Simply use `import scanpy as sc` and `import scanpy.external as sce` instead.

  FutureWarning,
/usr/local/anaconda3/envs/besca/lib/python3.7/site-packages/sklearn/externals/six.py:31: FutureWarning:

The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).



scanpy==1.4.6 anndata==0.7.1 umap==0.3.10 numpy==1.18.2 scipy==1.4.1 pandas==1.0.3 scikit-learn==0.22.2.post1 statsmodels==0.11.1 python-igraph==0.8.0 louvain==0.6.1


In [2]:
#import gene expression data
adata_gex = bc.Import.read_mtx('extracted_data/gene_expression/')

#import CITEseq
adata_CITEseq = bc.Import.read_mtx('extracted_data/CITEseq/')

reading matrix.mtx
... reading from cache file cache/extracted_data-gene_expression-matrix.h5ad
adding genes
adding cell barcodes
making var_names unique
adding annotation
reading matrix.mtx
... reading from cache file cache/extracted_data-CITEseq-matrix.h5ad
adding genes
adding cell barcodes
making var_names unique
adding annotation


In [3]:
adata_gex.obs = pd.read_csv('extracted_data/gene_expression/metadata.tsv', sep = '\t')

In [4]:
#reformat adata.var to match proper settings for 10X generated CITEseq data
adata_gex.obs = adata_gex.obs.drop(columns = ['orig.ident', 'dmx_hto_match', 'joint_classification_global', 'DMX_GLOBAL_BEST', 'hto_classification_global', 'nGene', 'nUMI', 'pctMT'])
adata_gex.obs['CELL'] = ['10X_CiteSeq_donor' + str(x) +'.'+ str(y) for x,y in zip(adata_gex.obs.get('sampleid'), adata_gex.obs.get('DEMUXLET.BARCODE'))]
adata_gex.obs['CONDITION'] = 'PBMC_healthy'
adata_gex.obs['sample_type'] = 'PBMC'
adata_gex.obs['donor'] = ['donor' + str(x) for x in adata_gex.obs.get('sampleid')]
adata_gex.obs = adata_gex.obs.get(['CELL', 'CONDITION', 'sample_type', 'donor', 'tenx_lane', 'cohort', 'batch', 'sampleid', 'timepoint'])
adata_gex.obs_names = adata_gex.obs['CELL']

#update cell barcodes to properly reflect annotation
adata_gex.obs_names = adata_gex.obs['CELL']
adata_gex.obs_names_make_unique() #make unique
adata_gex.obs['CELL'] = adata_gex.obs_names #write back to obs

In [5]:
#update adata.var object to reflect BESCA standards
adata_gex.var['SYMBOL'] = adata_gex.var.index.tolist()
adata_gex.var['ENSEMBL'] = 'N.A.'

adata_CITEseq.var['SYMBOL'] = adata_CITEseq.var.index.tolist()
adata_CITEseq.var['ENSEMBL'] = 'N.A.'

adata_gex.var['feature_type'] = 'Gene Expression'
adata_CITEseq.var['feature_type'] = 'Antibody Capture'

In [6]:
#generate new anndataobject that contains all information
from scipy.sparse import hstack, csr_matrix
X_merged = hstack([adata_gex.X, adata_CITEseq.X])

adata_merged = sc.AnnData(csr_matrix(X_merged)) #make sure to save as CSR matrix otherwise certain AnnData functions dont work
adata_merged.obs = adata_gex.obs
adata_merged.obs_names = adata_gex.obs_names
adata_merged.var = pd.concat([adata_gex.var, adata_CITEseq.var])
adata_merged.var_names = adata_merged.var.index.tolist()

In [7]:
adata_merged.var

,SYMBOL,ENSEMBL,feature_type
MIR1302-10,MIR1302-10,N.A.,Gene Expression
FAM138A,FAM138A,N.A.,Gene Expression
OR4F5,OR4F5,N.A.,Gene Expression
RP11-34P13.7,RP11-34P13.7,N.A.,Gene Expression
RP11-34P13.8,RP11-34P13.8,N.A.,Gene Expression
...,...,...,...
CD80_PROT,CD80_PROT,N.A.,Antibody Capture
CD86_PROT,CD86_PROT,N.A.,Antibody Capture
CD183_PROT,CD183_PROT,N.A.,Antibody Capture
CD34_PROT,CD34_PROT,N.A.,Antibody Capture


In [8]:
#write out dataobjects to file
adata_gex.write('adata_gene_raw.h5ad')
adata_CITEseq.write('adata_prot_raw.h5ad')
adata_merged.write('adata_raw.h5ad')

... storing 'CONDITION' as categorical
... storing 'sample_type' as categorical
... storing 'donor' as categorical
... storing 'tenx_lane' as categorical
... storing 'cohort' as categorical
... storing 'timepoint' as categorical
... storing 'ENSEMBL' as categorical
... storing 'feature_type' as categorical
... storing '"orig.ident"' as categorical
... storing '"tenx_lane"' as categorical
... storing '"cohort"' as categorical
... storing '"batch"' as categorical
... storing '"hash_maxID"' as categorical
... storing '"hash_secondID"' as categorical
... storing '"hto_classification"' as categorical
... storing '"hto_classification_global"' as categorical
... storing '"hash_ID"' as categorical
... storing '"adjmfc.time"' as categorical
... storing '"DMX_GLOBAL_BEST"' as categorical
... storing '"DEMUXLET.BARCODE"' as categorical
... storing '"sample"' as categorical
... storing '"sampleid"' as categorical
... storing '"joint_classification_global"' as categorical
... storing '"dmx_hto_matc

In [9]:
#write out to matrix and mtx files as well
bc.export.X_to_mtx(adata_merged, outpath='raw')
adata_merged.obs.to_csv('raw/metadata.tsv', sep = '\t')

writing out matrix.mtx ...
adata.X successfully written to matrix.mtx. 
feature annotation is present and will be written out
genes successfully written out to genes.tsv
cellbarcodes successfully written out to barcodes.tsv
